In [1]:
import keras
import tensorflow
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv")

In [3]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
df['quality'].value_counts()

quality
6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: count, dtype: int64

In [5]:
# Creating training and testing dataset
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Training data
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

# Test Data
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

# Validation data
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

data_signature = infer_signature(train_x, train_y)

In [6]:
# Normalizing target label for Sparse Categorical Cross Entropy

train_y -= 3
val_y -= 3
test_y -= 3

train_y = train_y.astype(np.float32)
val_y = val_y.astype(np.float32)
test_y = test_y.astype(np.float32)


In [7]:
print(train_x.shape)
print(val_x.shape)
print(test_x.shape)

(3134, 11)
(784, 11)
(980, 11)


In [ ]:
## Neural Net Model development

def train(params, epochs, train_x, train_y, val_x, val_y, test_x, test_y):

    ## Define Model Architecture

    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params['lr'], momentum=params['momentum']
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()] 
    )

    ## Train the ANN model with hyper paremetrs with MLFlow Tracking

    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=epochs, batch_size=32, verbose=0)


        # Evaluate Model

        eval_result = model.evaluate(test_x, test_y, batch_size=16, verbose=0)

        eval_rsme = eval_result[1]
        # eval_accuracy = eval_result[1]

        # Log the params and results

        mlflow.log_params(params)
        mlflow.log_metric("eval_rsme", eval_rsme)

        # log the model

        mlflow.tensorflow.log_model(model, "model", signature = data_signature)

        return {"loss": eval_result[0], "status": STATUS_OK, "model": model}

In [9]:
def objective(params):
    result = train(
        params,
        epochs=5,
        train_x=train_x,
        train_y=train_y,
        val_x=val_x,
        val_y=val_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [10]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

In [11]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hperopt

    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, model

    mlflow.log_params(best)
    mlflow.log_metric("RMSE", best_run["loss"])
    best_model_info = mlflow.tensorflow.log_model(best_run['model'], "best_model", signature=data_signature)

    # Print out the best parameters and loss

    print(f"Best Parameters: {best}")
    print(f"Best RMSE: {best_run['loss']}")

Epoch 1/5                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 46s 480ms/step - loss: 8.5480 - root_mean_squared_error: 2.9237
50/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.9639 - root_mean_squared_error: 2.8209   
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 7.3074 - root_mean_squared_error: 2.6995 - val_loss: 4.5322 - val_root_mean_squared_error: 2.1187

Epoch 2/5                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 4.7186 - root_mean_squared_error: 2.1722
61/98 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 3.9696 - root_mean_squared_error: 1.9916
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.7684 - root_mean_squared_error: 1.9396 - val_loss: 2.7746 - val_root_mean_squared_error: 1.6580

Epoch 3/5                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 2.5886 - root_mean_squared_error: 1.6089
66/98 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: 2.3004 - root_mean_squared_error: 1.5160
98/

2025/07/08 20:53:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmphpwgjd9a\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 



Epoch 1/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 38s 394ms/step - loss: 11.6114 - root_mean_squared_error: 3.4076
61/98 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - loss: 3.9385 - root_mean_squared_error: 1.9195  
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.0834 - root_mean_squared_error: 1.6844 - val_loss: 0.8173 - val_root_mean_squared_error: 0.8950

Epoch 2/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.9480 - root_mean_squared_error: 0.9737
59/98 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step - loss: 0.7583 - root_mean_squared_error: 0.8706
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7547 - root_mean_squared_error: 0.8686 - val_loss: 0.7075 - val_root_mean_squared_error: 0.8315

Epoch 3/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.7457 - root_mean_squared_error: 0.8636
61/98 ━━━━━━━

2025/07/08 20:53:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpkraq301s\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 



Epoch 1/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 37s 391ms/step - loss: 10.0041 - root_mean_squared_error: 3.1629
60/98 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 4.6125 - root_mean_squared_error: 2.0827  
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.5673 - root_mean_squared_error: 1.8133 - val_loss: 0.8901 - val_root_mean_squared_error: 0.9350

Epoch 2/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.9834 - root_mean_squared_error: 0.9917
57/98 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - loss: 0.8563 - root_mean_squared_error: 0.9251
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8266 - root_mean_squared_error: 0.9088 - val_loss: 0.7462 - val_root_mean_squared_error: 0.8537

Epoch 3/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.8615 - root_mean_squared_error: 0.9282
70/98 ━━━━━━━

...

Epoch 1/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 33s 345ms/step - loss: 8.0223 - root_mean_squared_error: 2.8324
65/98 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - loss: 7.3692 - root_mean_squared_error: 2.7118 
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.8093 - root_mean_squared_error: 2.6030 - val_loss: 3.3037 - val_root_mean_squared_error: 1.8094

Epoch 2/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 1.9153 - root_mean_squared_error: 1.3839
53/98 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - loss: 2.8269 - root_mean_squared_error: 1.6799
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6493 - root_mean_squared_error: 1.6258 - val_loss: 1.9225 - val_root_mean_squared_error: 1.3803

Epoch 3/5                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 2.3858 - root_mean_squared_error: 1.5446
29/98 ━━━━━━━━━

2025/07/08 20:53:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpdktfvpip\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 



100%|██████████| 4/4 [00:29<00:00,  7.28s/trial, best loss: 0.6038585901260376]


2025/07/08 20:53:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpb0nyt1sh\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


Best Parameters: {'lr': 0.005731888733425358, 'momentum': 0.3924356625688369}
Best RMSE: 0.6038585901260376
